In [42]:
import requests
import json
import os
from dotenv import load_dotenv

load_dotenv()
GEMINI_API_KEY = os.getenv("gemeniKey", "")

from google import genai
from google.genai import types
from PIL import Image
from io import BytesIO
import base64

def genIMG(content):
    client = genai.Client(api_key=GEMINI_API_KEY)
    contents = (content)
    response = client.models.generate_content(
        model="gemini-2.0-flash-preview-image-generation",
        contents=contents,
        config=types.GenerateContentConfig(
        response_modalities=['TEXT','IMAGE']
        )
    )
    for part in response.candidates[0].content.parts:
        if part.text is not None:
            print(part.text)
        elif part.inline_data is not None:
            image = Image.open(BytesIO((part.inline_data.data)))
            image.save('gemini-native-image.png')
            image.show()

if __name__ =='__main__':
    tt = genIMG("generate an image of YouTubeショート動画の台本を作成してください。テーマは「college」です。")

In [26]:
import requests
import json
import os
from dotenv import load_dotenv
import base64

load_dotenv()
TTS_API_KEY = os.getenv("gemeniKey", "")

from google import genai
from google.genai import types
import wave


import contextlib
import wave
from IPython.display import Audio

file_index = 0

@contextlib.contextmanager
def wave_file(filename, channels=1, rate=24000, sample_width=2):
    with wave.open(filename, "wb") as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(sample_width)
        wf.setframerate(rate)
        yield wf

def play_audio_blob(blob):
  global file_index
  file_index += 1

  fname = f'audio_{file_index}.wav'
  with wave_file(fname) as wav:
    wav.writeframes(blob.data)

  return Audio(fname, autoplay=True)

def play_audio(response):
    return play_audio_blob(response.candidates[0].content.parts[0].inline_data)

client = genai.Client(api_key=TTS_API_KEY)

MODEL_ID="gemini-2.5-flash-preview-tts"
voice_name = "kore" # @param ["Zephyr", "Puck", "Charon", "Kore", "Fenrir", "Leda", "Orus", "Aoede", "Callirhoe", "Autonoe", "Enceladus", "Iapetus", "Umbriel", "Algieba", "Despina", "Erinome", "Algenib", "Rasalgethi", "Laomedeia", "Achernar", "Alnilam", "Schedar", "Gacrux", "Pulcherrima", "Achird", "Zubenelgenubi", "Vindemiatrix", "Sadachbia", "Sadaltager", "Sulafar"]
#https://ai.google.dev/gemini-api/docs/speech-generation#voices

response = client.models.generate_content(
  model=MODEL_ID,
  contents="""read this in japanese:
  こんにちは、私の名前はジェめないです
  """,
  config={
      "response_modalities": ['Audio'],
      "speech_config": {
          "voice_config": {
              "prebuilt_voice_config": {
                  "voice_name": voice_name
              }
          }
      }
  },
)

play_audio(response)
     
    
data = response
print(data)
blob = response.candidates[0].content.parts[0].inline_data
play_audio_blob(blob)

candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=Blob(display_name=None, data=b'\x03\x00\xf8\xff\x00\x00\n\x00\x14\x007\x00V\x00l\x00\x88\x00\x86\x00j\x00\x1f\x00\xdc\xff\xae\xff\x84\xff\x8d\xff\xa2\xff\xae\xff\xd8\xff\xe4\xff\xdd\xff\xdc\xff\xd4\xff\xd3\xff\xd1\xff\xc9\xff\xae\xffy\xffl\xff\x83\xff\x95\xff\xc5\xff\xe3\xff\x02\x00#\x00-\x00\x0e\x00\xc0\xff\x83\xffW\xffN\xff{\xff\xd1\xff\x11\x00J\x00\\\x00J\x00*\x00\xf6\xff\xd6\xff\xc8\xff\xba\xff\xcd\xff\xd6\xff\xe0\xff\x11\x00C\x00|\x00\x90\x00Q\x00\xe0\xff\x94\xffm\xff|\xff\xa1\xff\xc5\xff\xd3\xff\xe4\xff\x07\x00(\x00P\x00f\x00A\x00\x03\x00\xd3\xff\xb5\xff\xb2\xff\xba\xff\xef\xff\x05\x00\x0b\x00E\x00f\x00^\x006\x00\xe9\xff\xb4\xff\xaf\xff\xd1\xff\r\x008\x00I\x00&\x00\xf7\xff\xf2\xff\xf1\xff4\x00\x98\x00\xb2\x00\x92\x00L\x00\xf3\xff\xbb\xff\xbc\xff\xed\xff$\x00I\x00c\x00h\x00V\x00U\x00W\x002\x00\x1e\x00\n\x00\xc8\xff\xa1\xff\x93\xff\xa2\xff\xe3\xff\x1e\x00G\x00K\x00\x08\x00\xb9\xffm\xff\

In [34]:
GEMINI_API_KEY = os.getenv("gemeniKey", "")

def generate_video_script(theme):
    """
    Generates a YouTube short video script based on a given theme.
    Returns a list of parsed statements from the script.
    """
    print(f"--- Generating script for theme: '{theme}'... ---")
    gemini_client = genai.Client(api_key=GEMINI_API_KEY)

    script_prompt = (f"Create a YouTube short video script about '{theme}'. "
        "Provide 3-5 very short, distinct sentences or phrases, each on a new line, "
        "suitable for a quick visual and narration. Format it as a numbered list. "
        "Example: '1. Student wakes up for class. 2. Rushing to campus. 3. Attending a lecture. 4. Studying late. 5. Celebrating graduation.'")
    
    response = gemini_client.models.generate_content(
    model="gemini-2.5-flash",
    contents=script_prompt
    )
    print(response.text)

    # Parse the generated script into individual statements
    script_lines = [line.strip() for line in response.text.split('\n') if line.strip() and line.strip()[0].isdigit()]
    statements = [line[line.find('.') + 1:].strip() for line in script_lines if line.find('.') != -1]
    
    return statements


response = generate_video_script("a day in the life of a software engineer")
print(response)

--- Generating script for theme: 'a day in the life of a software engineer'... ---
Here's a YouTube short script for 'a day in the life of a software engineer':

1.  Morning coffee & stand-up.
2.  Deep focus: coding and debugging.
3.  Team syncs & code reviews.
4.  Solving complex problems.
5.  Pushing code, then signing off!
['1.  Morning coffee & stand-up.', '2.  Deep focus: coding and debugging.', '3.  Team syncs & code reviews.', '4.  Solving complex problems.', '5.  Pushing code, then signing off!']
['Morning coffee & stand-up.', 'Deep focus: coding and debugging.', 'Team syncs & code reviews.', 'Solving complex problems.', 'Pushing code, then signing off!']


In [32]:
print(response)

['', '', '', '', '', '', '', '', '']
